# Hloubkové senzory - měření objektů
Cvičení je zaměřené na práci s [hloubkovými mapami](https://en.wikipedia.org/wiki/Depth_map) vzniklých použitím tzv. hloubkových senzorů. V laboratoři jsou k dispozici hloubkové senzory založené na několika principech zisku hloubkových map. Jedná se o kamery typu Kinect, RealSense nebo Time-of-Flight (ToF). Souhrně se často označují jako RGBD kamery, kde k standardní barevné informaci RGB přibyde informace hloubková "D" (depth).

Vytvořit stereokameru lze i pomocí dvou obyčejných kamer. Podrobný návod pro zájemce je k dispozici na příklad [zde](https://erget.wordpress.com/2014/02/01/calibrating-a-stereo-camera-with-opencv/).


---

## Typy systémů
<table>
    <tr>
        <th align="left" width="16%"><a href="https://en.wikipedia.org/wiki/Kinect">Microsoft Kinect</a></th>
        <th align="left" width="16%"><img src="images/kinect.png" width="50%"/></th>
        <th align="left" width="16%"><a href="https://en.wikipedia.org/wiki/Intel_RealSense">Intel RealSense</a></th>
        <th align="left" width="16%"><img src="images/realsense.png" width="50%"/></th>
        <th align="left" width="16%"><a href="https://en.wikipedia.org/wiki/Time-of-flight_camera">Basler ToF</a></th>
        <th align="left" width="16%"><img src="images/tof.png" width="50%"/></th>       
    </tr>
    <tr>
        <td align="left" colspan="2">Postaven na principu promítání laserového vzoru a měření jeho deformace infračervenou kamerou.</td>
        <td align="left" colspan="2">Využívá stejného principu jako Kinect, pouze má jiný vzor.</td>
        <td align="left" colspan="2">Využívá promítání světla z více LED zdrojů a měří čas jeho návratu (odrazu).</td>
    </tr>
    <tr>
        <td align="left" colspan="2">Je náchylný na měření venku. Denní světlo kazí obraz.</td>
        <td align="left" colspan="2">Měl by být vhodný i pro měření venku.</td>
        <td align="left" colspan="2">Je náchylný na měření venku. Denní světlo kazí obraz.</td>
    </tr>
    <tr>
        <td align="left" colspan="2">Více Kinectů vedle sebe si vzájemně kazí vzory.</td>
        <td align="left" colspan="2">Nejnovější a nejmenší senzor na trhu.</td>
        <td align="left" colspan="2">Průmyslová konstrukce pro běh 24/7.</td>
    </tr>
</table>

---

## Využití

Kromě zjevného využití RGBD kamer k měření vzdálenosti objektu od senzoru (např. precizní robotická manipulace, 3D skeny, výpočet objemu součástky) jsou kamery užitečné i pro robustnější detekce a klasifikace objektů.

**Příklad:**

Obrázek vlevo nasnímaný RGBD kamerou představuje naměřenou hloubku pomocí barevné škály.
Nelze však jednoduše určit, zda se jedná o jablko nebo pomeranč. Je potřeba použít dodatečnou barevnou informaci.

Naopak samotná 2D kamera by nebyla schopna rozpoznat rozdíl mezi plochou nálepkou ovoce a skutečným hmotným ovocem.

To umožňuje např. detekovat hloubku jablka ve stromu a zároveň jeho zralost dle barvy.

Hloubková informace v synergii s tou barevnou tak poskytuje nové souvislosti v obrazových datech a umožňuje pokročilejší analýzy, nikoliv jen měření vzdáleností.

<img src="images/tof_advantage_showcase.png" alt="RGBD" width="600"/>

---

## Úkol
Cílem cvičení je využít hloubkovou mapu ze senzoru **Basler ToF** pro detekci zkoumaného objektu - zavěšené dřevěné koule mimikující jablko. Dále je potřeba vyfiltrovat dřevěnou krychli, kterou měřit nechceme a výstup vhodně vizualizovat.

Úloha je inspirovaná řešením z aplikovaného výzkumu samosběrného robota [[1]](https://robomechjournal.springeropen.com/articles/10.1186/s40648-022-00230-y):

<table>
    <tr>
        <td><img src="images/fruit_picking_robot.png" alt="Fruit picking robot" height="260"/></td>
        <td><img src="images/pears_3D.png" alt="Pears 3D" height="260"/></td>
        <td><img src="images/robot_action.png" alt="Robot in action" height="260"/></td>
    </tr>
</table>


---

### Import knihoven a konfigurace

In [1]:
import os
import io
import builtins


import cv2
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patheffects import withStroke

from improutils import *

%matplotlib inline
np.set_printoptions(formatter={'float': lambda x: "{0:0.3f}".format(x)})

---

#### 1) Nasnímejte data hloubkovou kamerou Basler ToF

Pro práci s hloubkovou kamerou využijte software Basler ToF Viewer. Kameru připojte k PC, nasnímejte zkoumaný objekt a výstup uložte. Typ souboru při ukládání vyberte jako "Points intensity" (končí příponou points-intensity.pcd).

Dbejte na to, aby při pořizování dat byla kamera **nastavena rovnoběžně s pozadím**, a to bez rušivých elementů v záběru, např. okraj monitoru apod. 

Usnadnit snímání Vám může nastavení "Maximum depth [mm]" v programu, který vyfiltruje vše, co je dále, než zvolený threshold.

---

#### 2) Načtěte data ve formátu point cloud `.pcd (text)` a z dat získejte hloubkovou mapu. Mapu zobrazte.

##### a) Zobrazte strukturu .pcd dat

In [ ]:
with builtins.open(..., 'r') as f:  ### soubor s pcd daty
    pcd = f.readlines()

header = pcd[:11]
data = pcd[11:]

for line in ... : # zobrazení hlavičky
    print(line)

for line in ... : # zobrazení několika bodů
    print(line)

##### b) Převeďte .pcd data na hloubkovou mapu

Informace v hlavičce .pcd souboru vám mimo jiné prozradila, že data jsou ve formátu 4 float hodnot x, y, z, rgb. Při pořizování dat byla kamera nastavena rovnoběžně s pozadím, tudíž nás budou zajímat pouze **_z_** hodnoty -> hloubková mapa.

Souřadnice ```z = 0``` je dle výrobce na přední straně krytu kamery a od ní dále se měří vzdálenost. Pixely s nespolehlivými údaji o vzdálenosti (např. poblíž hran objektů) nebo pixely, které představují objekty v oblastech mimo ROI (vámi nastavená maximální hloubka), mají hodnotu NaN.

In [140]:
# Funkce pro převod .pcd dat do hloubkové mapy o velikosti height x width
def pcd_to_depth(pcd, height, width):
    data = pcd
    data = [float(x.split(' ')[2]) for x in data] # extrakce hloubkové hodnoty
    data = np.reshape(data, (height, width))
    return data

In [141]:
depths = pcd_to_depth(..., ..., ...)  ###

##### c) Normalizujte a zobrazte data
Pro využití automatické segmentace je třeba data převézt na typ uint8. Pro převedení dat do rozsahu [0, 255] je třeba použít normalizaci, OpenCV a improutils obsahují vhodnou funkci.

Normalizovaná data zobrazte pomocí čtyř různých [colormap](https://matplotlib.org/stable/users/explain/colors/colormaps.html), vhodných pro hloubková data.

In [2]:
def subplot_image(image, subplots=(2, 2), cmaps=['gray'], normalize=False, ticks_off=True, title_size=12):
    """
    Plots the same image in multiple subplots with different color maps.

    Parameters
    ----------
    image : ndarray
        Image to be shown.
    subplots : tuple
        Shape of the subplot grid (rows, columns).
    cmaps : list
        List of colormaps to be used for each subplot.
    normalize : bool
        If True, image will be normalized.
    ticks_off : bool
        If True, axis decorations will be hidden.
    title_size : int
        Size of the title.

    Returns
    -------
    None
    """
    rows, cols = subplots
    fig, axes = plt.subplots(rows, cols, figsize=(cols * 6, rows * 6))
    
    # Flatten axes array if it's 2D for easier iteration
    axes = axes.ravel()

    # Determine number of subplots and iterate
    num_subplots = min(len(axes), len(cmaps))
    for i in range(num_subplots):
        ax = axes[i]
        if ticks_off:
            ax.axis('off')

        # Set colormap and display the image
        cmap = cmaps[i] if i < len(cmaps) else 'gray'
        norm = Normalize() if normalize else NoNorm()
        
        ax.imshow(image, cmap=plt.get_cmap(cmap), norm=norm)
        ax.set_title(f'Cmap: {cmap}', fontsize=title_size)

    # Hide any unused subplots if there are fewer cmaps than grid cells
    for j in range(num_subplots, len(axes)):
        fig.delaxes(axes[j])

    plt.tight_layout()
    plt.show()

In [144]:
depths_normalized = ... ### normalizace dat

In [ ]:
subplot_image(..., subplots=(2, 2), cmaps=['gray', ...]) ### zobrazení s různými colormapy

#### 3) Segmentujte objekty
Při použití klasické segmentace není vhodné segmentovat normalizovaná data, jelikož by segmentace nebyla založena na reálných datech. Funkce pro automatické segmentování v improutils používá *Otsouvu metodu*, která u originálních a normalizovaných bude segmentovat stejně. 

Využijte **automatickou** segmentaci na **normalizovaných** datech a **manuální** pro **originální** i **normalizovaná** data. Výsledné 3 masky zobrazte.

Pro volbu vhodných hodnot při manuální segmentaci originálních hodnot si můžete pomoci např. matplotlib histogramem (pozor na NaN hodnoty) či podstatou, jaká data hloubková kamera zaznamenává.

Dále pracujte jen s nejlepším segmentace.

In [ ]:
img_seg_auto = ... ###
img_seg_org = ... ###
img_seg_norm = ... ###
plot_images(depths_normalized, ...) # porovnejte normalizovaný a segmentovaný obraz

---

#### 4) Získejte masku naměřených objektů

Pro měření objektů je třeba získat kontury objektů. Počet kontur by měl odpovídat počtu zavěšených objektů. Filtrací odstraníme nekruhové objekty a šum. 

Kontury vyfiltrujte minimální plochou a na základě jejich kulatosti.
Kulatost je definována vzorcem $\dfrac{4 * \pi * plocha}{obvod^2}$ [[2]](https://en.wikipedia.org/wiki/Roundness). Pro některé výpočty týkajících se vlastností kontury budete muset použít vhodné funkce z OpenCV2 či improutils.

Zobrazte novou masku na základě vyfiltrovaných kontur. Pro hledání kontur využijte jednu ze 3 vytvořených masek.



In [ ]:
contour_drawn, count, contours = ... ###
print(f"Počet nalezených kontur: {count}")

In [ ]:
filtered_image = np.zeros_like(...)  # prázdný obraz pro zápis přijatých kontur
passed_contours = []

# Filtrování kontur na základě jejich kulatosti
for i, ... in enumerate(...):
    area = ...       # obsah kontury
    perimeter = ... # obvod kontury
    
    circularity = ... # porovnání kulatosti kontury a ideálního kruhu
    
    accept = "NE" # flag pro výpis
    
    if ... < circularity < ...: # doplňte vhodné hodnoty pro odfiltrování nekulatých objektů 
       
        accept = "ANO"
        passed_contours.append(...)
        
    
    print(f"Kontura {i}: Plocha = {area:.2f}, Obvod = {perimeter:.2f}, Kulatost = {circularity:.4f}, Přijímáme = {accept}")

cv2.drawContours(...) # vykreslení všech přijatých kontur do jednoho obrazu   
plot_images(...)

---

#### 5) Slučte hloubkovou informaci s maskou vyfiltrovaných jablek
OpenCV obsahuje vhodnou funkci pro logické sloučení dvou obrazů. Výsledek vizualizujte pomocí vhodné colormapy.

In [ ]:
ball_depth = ... #
subplot_image(..., subplots=(2,1), cmaps=[...])

---

#### 6) Nalezněte koule, které jsou nejdále a nejblíže od kamery

##### a) Vizualizujte indexy přijatých kontur (koulí) na hloubkovém obrazu z předchozího kroku

In [ ]:
number_contours_img = ball_depth.copy()
for i, contour in enumerate(...):
    x, y, w, h = ... # vhodná funkce cv2 na obdélníkové ohraničení kontury
    number_contours_img = cv2.putText(number_contours_img, str(i) , (x, y), cv2.FONT_HERSHEY_SIMPLEX, 1, 255, 1, cv2.LINE_AA)
 
plot_images(...) ###

##### b) Vypište index koule visící nejblíže a nejdále od kamery
Nezapomeňte správně pracovat s NaN hodnotami.

In [ ]:
results = []

for contour in passed_contours:
    
    contour_mask = np.zeros(..., dtype=np.uint8) # příprava masky pro aplikaci na originální data
    contour_mask = cv2.drawContours(..., [...], -1, 255, -1) # vykreslení kontury do masky
    
    ... ### aplikace masky, výběr nejbližšího bodu a uložení
    
min_depth_index = ...
max_depth_index = ...

print('Nejblíže ke kameře je koule:  č.{}'.format(max_depth_index))
print('Nejdál ke kameře je koule:  č.{}'.format(min_depth_index))
    